## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
from ipywidgets.embed import embed_minimal_html
import gmaps
# Import API key
from config import g_key

gmaps.configure(api_key= g_key)

# Configure gmaps API key
from config import g_key

fig = gmaps.figure()
embed_minimal_html('export.html', views=[fig])

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Albany,US,42.6001,-73.9662,64.63,82,92,4.36,overcast clouds
1,1,Cape Town,ZA,-33.9258,18.4232,64.11,84,97,4.94,overcast clouds
2,2,Susanville,US,40.4163,-120.6530,70.81,27,1,1.01,clear sky
3,3,Jamestown,US,42.0970,-79.2353,69.39,80,90,8.01,overcast clouds
4,4,Butaritari,KI,3.0707,172.7902,81.39,68,40,9.48,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp)
                                      & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
4,4,Butaritari,KI,3.0707,172.7902,81.39,68,40,9.48,scattered clouds
10,10,Garowe,SO,8.4054,48.4845,80.56,63,97,12.97,overcast clouds
13,13,Georgetown,MY,5.4112,100.3354,78.89,84,20,0.00,few clouds
18,18,Vaini,TO,-21.2000,-175.2000,77.16,83,75,8.05,broken clouds
23,23,Nicoya,CR,10.1483,-85.4520,85.55,79,83,2.33,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        209
City           209
Country        209
Lat            209
Lng            209
Max Temp       209
Humidity       209
Cloudiness     209
Wind Speed     209
Description    209
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
4,4,Butaritari,KI,3.0707,172.7902,81.39,68,40,9.48,scattered clouds
10,10,Garowe,SO,8.4054,48.4845,80.56,63,97,12.97,overcast clouds
13,13,Georgetown,MY,5.4112,100.3354,78.89,84,20,0.00,few clouds
18,18,Vaini,TO,-21.2000,-175.2000,77.16,83,75,8.05,broken clouds
23,23,Nicoya,CR,10.1483,-85.4520,85.55,79,83,2.33,broken clouds
...,...,...,...,...,...,...,...,...,...,...
664,664,Sakakah,SA,29.9697,40.2064,81.97,18,3,6.85,clear sky
670,670,Sulangan,PH,11.1373,123.7228,83.01,77,85,22.53,overcast clouds
678,678,Beaufort,MY,5.3473,115.7455,78.96,94,99,3.00,overcast clouds
680,680,Zhuhai,CN,22.2769,113.5678,82.06,89,20,8.05,few clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Butaritari,KI,81.39,scattered clouds,3.0707,172.7902,
10,Garowe,SO,80.56,overcast clouds,8.4054,48.4845,
13,Georgetown,MY,78.89,few clouds,5.4112,100.3354,
18,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,
23,Nicoya,CR,85.55,broken clouds,10.1483,-85.4520,
27,Kapaa,US,82.72,light rain,22.0752,-159.3190,
29,Pacific Grove,US,83.57,clear sky,36.6177,-121.9166,
31,Alofi,NU,77.25,overcast clouds,-19.0595,-169.9187,
33,Victoria,HK,82.33,overcast clouds,22.2855,114.1577,
38,San Luis De Gaceno,CO,76.33,overcast clouds,4.8205,-73.1685,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
payload = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat= row["Lat"]
    lng= row["Lng"]
    
    payload["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=payload).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("No hotel was found ..... skipping")
        
hotel_df.head()
        

No hotel was found ..... skipping
No hotel was found ..... skipping
No hotel was found ..... skipping
No hotel was found ..... skipping
No hotel was found ..... skipping
No hotel was found ..... skipping
No hotel was found ..... skipping
No hotel was found ..... skipping
No hotel was found ..... skipping
No hotel was found ..... skipping
No hotel was found ..... skipping
No hotel was found ..... skipping


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Butaritari,KI,81.39,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
10,Garowe,SO,80.56,overcast clouds,8.4054,48.4845,Curubo Hotel
13,Georgetown,MY,78.89,few clouds,5.4112,100.3354,Cititel Penang
18,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
23,Nicoya,CR,85.55,broken clouds,10.1483,-85.4520,Las Cabinas Del Sueño


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(subset = ["Hotel Name"])

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Butaritari,KI,81.39,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
10,Garowe,SO,80.56,overcast clouds,8.4054,48.4845,Curubo Hotel
13,Georgetown,MY,78.89,few clouds,5.4112,100.3354,Cititel Penang
18,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
23,Nicoya,CR,85.55,broken clouds,10.1483,-85.4520,Las Cabinas Del Sueño
...,...,...,...,...,...,...,...
664,Sakakah,SA,81.97,clear sky,29.9697,40.2064,Fakhamat Al Orjoana Apartment
670,Sulangan,PH,83.01,overcast clouds,11.1373,123.7228,Maia's Beach Resort
678,Beaufort,MY,78.96,overcast clouds,5.3473,115.7455,The Gem Hotel Beaufort
680,Zhuhai,CN,82.06,few clouds,22.2769,113.5678,Nanyang Seascape Hotel Zhuhai Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Database/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
    <dt>Hotel Name</dt>
        <dd>{Hotel Name}</dd>
    <dt>City</dt>
        <dd>{City}</dd>
    <dt>Country</dt>
        <dd>{Country}</dd>
    <dt>Current Weather</dt>
        <dd>{Description} and {Max Temp}</dd>
<dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))